In [ ]:
# @title Install necessary libraries
!pip install openai ipywidgets graphviz ipython --quiet

# @title Load OpenAI API Credentials
from getpass import getpass
import os
import openai

# Check if key is already set (e.g., in Colab secrets)
api_key_env = os.environ.get('OPENAI_API_KEY')

if api_key_env:
    print("OpenAI API Key found in environment variables.")
    OPENAI_KEY = api_key_env
else:
    print("OpenAI API Key not found in environment variables.")
    # Fallback to getpass if not found
    try:
        OPENAI_KEY = getpass('Enter your OpenAI API Key: ')
        os.environ['OPENAI_API_KEY'] = OPENAI_KEY
        openai.api_key = OPENAI_KEY
        # Test the key with a simple call (optional but recommended)
        # client = openai.OpenAI()
        # client.models.list()
        print("OpenAI API Key configured.")
    except Exception as e:
        print(f"Failed to configure OpenAI API Key: {e}")
        OPENAI_KEY = None # Ensure it's None if setup failed

# Configure the OpenAI client (using the newer openai > 1.0 library structure)
if OPENAI_KEY:
    try:
      client = openai.OpenAI(api_key=OPENAI_KEY)
      print("OpenAI client initialized successfully.")
    except Exception as e:
      print(f"Error initializing OpenAI client: {e}")
      client = None
else:
    print("Cannot initialize OpenAI client: API Key is missing.")

# @title Import other required libraries
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import graphviz
import uuid # For generating unique node IDs
import json # For potentially storing/loading graphs
import time # For simulating delays if needed
import traceback # For better error reporting
import copy # For deep copying node data if needed

In [ ]:
# @title Node Graph Data Structure and LLM Function

# Global dictionary to store our nodes and their connections
# Structure Change: 'outputs' values are now LISTS of node IDs
# nodes = {
#     'node_id_1': {
#         'name': 'User Friendly Name', 'type': 'Button', 'options': {},
#         'outputs': {'on_click': ['node_id_2', 'node_id_4']} # <<< LIST HERE
#     },
#     'node_id_2': {
#         'name': 'Ask LLM', 'type': 'LLM Program', 'options': {'prompt': '...'},
#         'outputs': {'result': ['node_id_3']} # <<< LIST HERE
#      },
#      'node_id_3': {
#         'name': 'Show Result', 'type': 'Show', 'options': {'text_template': '...'},
#         'outputs': {} # No outputs from Show node
#      },
#      'node_id_4': {
#          'name': 'Show Another', 'type': 'Show', 'options': {'text_template': 'Button also triggered this!'},
#          'outputs': {}
#      }
# }
nodes = {}
editing_node_id = None # Track which node ID is currently being edited

# --- LLM Interaction Function (No Change) ---
def call_llm(prompt_text, model="gpt-3.5-turbo"):
    """Calls the OpenAI API to get a completion."""
    if not client:
        return "Error: OpenAI client not initialized. Please check API Key."
    try:
        # print(f"\nDEBUG: Calling LLM with prompt: '{prompt_text[:100]}...'") # Debug print
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant executing user instructions."},
                {"role": "user", "content": prompt_text}
            ]
        )
        result = response.choices[0].message.content.strip()
        # print(f"DEBUG: LLM Response received: '{result[:100]}...'") # Debug print
        return result
    except Exception as e:
        error_message = f"Error calling OpenAI API: {e}"
        print(f"DEBUG: {error_message}") # Debug print
        return f"Error: Could not get response from LLM. Details: {e}"

print("Node Graph Data Structure (with list outputs) and LLM function defined.")

In [ ]:
# @title Graph Visualization Function (Handles Multiple Connections)

graph_output = widgets.Output()

def visualize_graph():
    """Renders the current node graph using Graphviz, handling multiple outputs."""
    dot = graphviz.Digraph(comment='Node Graph', format='png')
    dot.attr(rankdir='LR')

    with dot.subgraph(name='cluster_nodes') as c:
        c.attr(label='Nodes', style='filled', color='lightgrey')
        for node_id, node_data in nodes.items():
            node_label = f"ID: {node_id}\nType: {node_data['type']}\nName: {node_data['name']}"
            # Add options preview if applicable
            if node_data['type'] == 'LLM Program':
                node_label += f"\nPrompt: {node_data['options'].get('prompt', '')[:30]}..."
            elif node_data['type'] == 'Show':
                 node_label += f"\nText: {node_data['options'].get('text_template', '')[:30]}..."

            # Highlight node being edited
            node_color = 'lightblue' if node_id == editing_node_id else 'white'
            c.node(node_id, label=node_label, shape='box', style='filled', fillcolor=node_color)

    # Add edges after all nodes are defined
    for node_id, node_data in nodes.items():
        for port_name, target_node_ids in node_data.get('outputs', {}).items():
            # Ensure target_node_ids is a list (even if loaded from old format)
            if isinstance(target_node_ids, str):
                target_node_ids = [target_node_ids] # Convert old format on the fly

            if isinstance(target_node_ids, list):
                for target_node_id in target_node_ids: # Iterate through the list
                    if target_node_id in nodes:
                        dot.edge(node_id, target_node_id, label=port_name)
                    else:
                        print(f"Warning: Connection from {node_id} ({port_name}) points to non-existent node {target_node_id}")
            # else: Handle potential data corruption? print warning?

    with graph_output:
        clear_output(wait=True)
        if not nodes:
            print("Graph is empty. Add some nodes!")
        else:
            try:
                display(dot)
            except graphviz.backend.execute.ExecutableNotFound:
                 display(Markdown("**Error:** `graphviz` executable not found. Rendering failed."))
            except Exception as e:
                 display(Markdown(f"**Error rendering graph:** {e}"))

print("Graph Visualization function updated for multiple connections.")
# Initial visualization (will be empty)
visualize_graph()

In [ ]:
# @title Node Definition, Editing, and Removal UI

import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import uuid
import graphviz # Ensure graphviz is imported if not already globally

# --- Global State ---
if 'nodes' not in globals(): nodes = {}
editing_node_id = None # Track which node ID is currently being edited

# --- Output Widgets ---
edit_feedback_output = widgets.Output()
add_node_output = widgets.Output() # Combined output for add/update actions
add_connection_output = widgets.Output()
if 'graph_output' not in globals(): graph_output = widgets.Output()
if 'visualize_graph' not in globals():
    def visualize_graph(): print("Error: visualize_graph not defined") # Placeholder

# --- Helper: Get Node Choices for Dropdowns ---
def get_node_choices():
    """Returns a dictionary of 'Display Name': node_id for dropdowns."""
    choices = {f"{data['name']} ({node_id})": node_id for node_id, data in nodes.items()}
    if not choices:
        return {"(No nodes available)": None}
    return choices

# --- Helper Functions for Dropdowns and Ports ---
def update_all_dropdowns():
    """Updates all node selection dropdowns."""
    global source_node_dropdown, target_node_dropdown, select_node_edit_dropdown # Make them accessible

    node_choices = get_node_choices()
    current_source = source_node_dropdown.value
    current_target = target_node_dropdown.value
    current_edit_selection = select_node_edit_dropdown.value

    # Update options
    source_node_dropdown.options = node_choices
    target_node_dropdown.options = node_choices
    select_node_edit_dropdown.options = node_choices

    # Try to restore selections
    source_node_dropdown.value = current_source if current_source in node_choices.values() else (list(node_choices.values())[0] if node_choices else None)
    target_node_dropdown.value = current_target if current_target in node_choices.values() else (list(node_choices.values())[0] if node_choices else None)
    select_node_edit_dropdown.value = current_edit_selection if current_edit_selection in node_choices.values() else (list(node_choices.values())[0] if node_choices else None)

    update_source_ports(None) # Update ports based on potentially changed source node

def update_source_ports(change):
    """Update the source port dropdown based on the selected source node."""
    if 'source_port_dropdown' not in globals(): return # Widget not ready
    selected_node_id = source_node_dropdown.value
    port_options = []
    if selected_node_id and selected_node_id in nodes:
        node_type = nodes[selected_node_id]['type']
        if node_type == 'Button': port_options = ['on_click']
        elif node_type == 'LLM Program': port_options = ['result']
        # 'Show' nodes have no output ports in this model
    source_port_dropdown.options = port_options
    source_port_dropdown.value = port_options[0] if port_options else None


# --- Widgets for Node Creation/Editing ---
node_type_dropdown = widgets.Dropdown(options=['Button', 'LLM Program', 'Show'], value='Button', description='Node Type:', style={'description_width': 'initial'})
node_name_text = widgets.Text(value='My Node', placeholder='Enter a descriptive name', description='Node Name:', style={'description_width': 'initial'})

button_options_box = widgets.VBox([])
llm_options_box = widgets.VBox([widgets.Textarea(value='', placeholder='Enter LLM prompt...', description='LLM Prompt:', layout={'width': '95%'}, style={'description_width': 'initial'})])
show_options_box = widgets.VBox([widgets.Textarea(value='{input}', placeholder='Enter text. Use {input} for data...', description='Show Text:', layout={'width': '95%'}, style={'description_width': 'initial'})])

options_accordion = widgets.Accordion(children=[button_options_box, llm_options_box, show_options_box], selected_index=0)
options_accordion.set_title(0, 'Button Options'); options_accordion.set_title(1, 'LLM Options'); options_accordion.set_title(2, 'Show Options')

add_update_node_button = widgets.Button(description='Add Node', button_style='success', icon='plus')
cancel_edit_button = widgets.Button(description='Cancel Edit', button_style='warning', icon='times', visible=False) # Initially hidden

# --- Widgets for Node Selection & Management ---
select_node_edit_dropdown = widgets.Dropdown(description='Select Node:', style={'description_width': 'initial'})
load_node_button = widgets.Button(description='Load for Edit', button_style='info', icon='edit')
remove_node_button = widgets.Button(description='Remove Selected Node', button_style='danger', icon='trash')

# --- Widgets for Adding Connections ---
source_node_dropdown = widgets.Dropdown(description='Source Node:', style={'description_width': 'initial'})
source_port_dropdown = widgets.Dropdown(description='Output Port:', style={'description_width': 'initial'})
target_node_dropdown = widgets.Dropdown(description='Target Node:', style={'description_width': 'initial'})
add_connection_button = widgets.Button(description='Add Connection', button_style='primary', icon='link')


# --- Event Handlers ---

def reset_node_form(clear_name=True):
    """Resets the node creation form to default."""
    global editing_node_id
    editing_node_id = None
    node_type_dropdown.value = 'Button'
    if clear_name: node_name_text.value = 'My Node' # Reset or keep name? Resetting is cleaner.
    # Reset options (find the textareas and clear them)
    llm_ta = next((w for w in llm_options_box.children if isinstance(w, widgets.Textarea)), None)
    if llm_ta: llm_ta.value = ''
    show_ta = next((w for w in show_options_box.children if isinstance(w, widgets.Textarea)), None)
    if show_ta: show_ta.value = '{input}'
    options_accordion.selected_index = 0
    add_update_node_button.description = 'Add Node'
    add_update_node_button.button_style = 'success'
    add_update_node_button.icon = 'plus'
    cancel_edit_button.visible = False
    node_type_dropdown.disabled = False # Re-enable type change for adding
    visualize_graph() # Redraw graph to remove edit highlight

def on_cancel_edit_clicked(b):
    """Resets the form when Cancel Edit is clicked."""
    with add_node_output:
        clear_output(wait=True)
        print("Edit cancelled.")
    reset_node_form()

def handle_node_type_change(change):
    """Sync accordion and suggest name on type change, only if NOT editing."""
    if editing_node_id is None: # Only act if we are adding a new node
        type_to_index = {'Button': 0, 'LLM Program': 1, 'Show': 2}
        options_accordion.selected_index = type_to_index.get(change.new, 0)
        if change.new == 'LLM Program': node_name_text.value = 'Ask LLM'
        elif change.new == 'Show': node_name_text.value = 'Display Result'
        elif change.new == 'Button': node_name_text.value = 'Start Action'

def on_add_update_node_clicked(b):
    """Adds a new node or updates the currently editing node."""
    global editing_node_id
    node_type = node_type_dropdown.value
    node_name = node_name_text.value.strip() or f"Untitled_{node_type}"

    if not node_name:
         with add_node_output:
            clear_output(wait=True)
            print("❌ Error: Node name cannot be empty.")
         return

    options = {}
    if node_type == 'LLM Program':
        llm_ta = next((w for w in llm_options_box.children if isinstance(w, widgets.Textarea)), None)
        if llm_ta: options['prompt'] = llm_ta.value
    elif node_type == 'Show':
        show_ta = next((w for w in show_options_box.children if isinstance(w, widgets.Textarea)), None)
        if show_ta: options['text_template'] = show_ta.value

    with add_node_output:
        clear_output(wait=True)
        if editing_node_id: # --- UPDATE EXISTING NODE ---
            if editing_node_id not in nodes:
                 print(f"❌ Error: Node {editing_node_id} not found for update (maybe removed?).")
                 reset_node_form()
                 return

            print(f"Updating node '{nodes[editing_node_id]['name']}' ({editing_node_id})...")
            # NOTE: We are NOT allowing type change during edit in this version for simplicity
            if nodes[editing_node_id]['type'] != node_type:
                 print(f"⚠️ Warning: Node type change during edit is not supported. Keeping original type: {nodes[editing_node_id]['type']}.")
                 node_type = nodes[editing_node_id]['type'] # Revert type

            nodes[editing_node_id]['name'] = node_name
            nodes[editing_node_id]['options'] = options
            # Don't touch 'outputs' during edit via this form
            print(f"✅ Node '{node_name}' ({editing_node_id}) updated.")
            reset_node_form(clear_name=False) # Keep name maybe? No, reset for consistency.
            reset_node_form(clear_name=True)

        else: # --- ADD NEW NODE ---
            new_node_id = str(uuid.uuid4())[:8]
            while new_node_id in nodes: new_node_id = str(uuid.uuid4())[:8] # Ensure unique ID

            nodes[new_node_id] = {
                'name': node_name, 'type': node_type, 'options': options, 'outputs': {}
            }
            print(f"✅ Node '{node_name}' ({new_node_id}) added.")
            reset_node_form(clear_name=True) # Clear form for next addition

    update_all_dropdowns()
    visualize_graph()

def on_load_node_clicked(b):
    """Loads the selected node's data into the form for editing."""
    global editing_node_id
    selected_id = select_node_edit_dropdown.value
    with edit_feedback_output:
        clear_output(wait=True)
        if not selected_id or selected_id not in nodes:
            print("❌ Please select a valid node to load.")
            reset_node_form() # Ensure form is reset if selection is invalid
            return

        print(f"Loading node '{nodes[selected_id]['name']}' ({selected_id}) for editing...")
        editing_node_id = selected_id
        node_data = nodes[selected_id]

        # Populate form widgets
        node_name_text.value = node_data['name']
        node_type_dropdown.value = node_data['type']
        node_type_dropdown.disabled = True # Prevent type change during edit for simplicity

        # Populate options
        type_to_index = {'Button': 0, 'LLM Program': 1, 'Show': 2}
        options_accordion.selected_index = type_to_index.get(node_data['type'], 0)

        if node_data['type'] == 'LLM Program':
            llm_ta = next((w for w in llm_options_box.children if isinstance(w, widgets.Textarea)), None)
            if llm_ta: llm_ta.value = node_data['options'].get('prompt', '')
        elif node_data['type'] == 'Show':
            show_ta = next((w for w in show_options_box.children if isinstance(w, widgets.Textarea)), None)
            if show_ta: show_ta.value = node_data['options'].get('text_template', '')

        # Update button states
        add_update_node_button.description = 'Update Node'
        add_update_node_button.button_style = 'info'
        add_update_node_button.icon = 'save'
        cancel_edit_button.visible = True

    visualize_graph() # Highlight the node being edited

def on_remove_node_clicked(b):
    """Removes the selected node and its connections."""
    global editing_node_id
    node_to_remove_id = select_node_edit_dropdown.value
    with edit_feedback_output:
        clear_output(wait=True)
        if not node_to_remove_id or node_to_remove_id not in nodes:
            print("❌ Please select a valid node to remove.")
            return

        removed_name = nodes[node_to_remove_id]['name']
        print(f"Removing node '{removed_name}' ({node_to_remove_id})...")

        # 1. Delete the node itself
        del nodes[node_to_remove_id]

        # 2. Remove incoming connections from other nodes
        nodes_to_check = list(nodes.keys()) # Iterate over a copy of keys
        for check_id in nodes_to_check:
             if check_id in nodes: # Check if node still exists (might have been removed in same loop?)
                outputs_to_check = list(nodes[check_id]['outputs'].items()) # Iterate over copy
                for port_name, target_ids in outputs_to_check:
                    # Ensure target_ids is a list
                    if isinstance(target_ids, str): target_ids = [target_ids]

                    if isinstance(target_ids, list) and node_to_remove_id in target_ids:
                        print(f"  Removing connection from {check_id} [{port_name}] -> {node_to_remove_id}")
                        nodes[check_id]['outputs'][port_name].remove(node_to_remove_id)
                        # Optional: clean up empty port lists
                        if not nodes[check_id]['outputs'][port_name]:
                             del nodes[check_id]['outputs'][port_name]


        print(f"✅ Node '{removed_name}' and its connections removed.")

        # If the removed node was being edited, cancel edit mode
        if editing_node_id == node_to_remove_id:
            print("  Cancelling edit mode as the node was removed.")
            reset_node_form()
        else:
             # Still need to redraw graph even if not cancelling edit
             visualize_graph()

    update_all_dropdowns() # Update dropdowns to remove the deleted node


def on_add_connection_clicked(b):
    """Adds a connection (appends to list) between two nodes."""
    source_id = source_node_dropdown.value
    port_name = source_port_dropdown.value
    target_id = target_node_dropdown.value

    with add_connection_output:
        clear_output(wait=True)
        # --- Input Validation ---
        if not source_id or not port_name or not target_id:
            print("❌ Error: Select valid source node, output port, and target node.")
            return
        if source_id == target_id:
            print("❌ Error: Cannot connect a node to itself.")
            return
        if source_id not in nodes or target_id not in nodes:
            print("❌ Error: Source or target node does not exist.")
            return
        # Check if port is valid for source node type (redundant with dropdown logic but safer)
        valid_ports = []
        if nodes[source_id]['type'] == 'Button': valid_ports = ['on_click']
        elif nodes[source_id]['type'] == 'LLM Program': valid_ports = ['result']
        if port_name not in valid_ports:
            print(f"❌ Error: Port '{port_name}' is not valid for node type '{nodes[source_id]['type']}'.")
            return

        # --- Add Connection (Append Logic) ---
        output_dict = nodes[source_id]['outputs']
        if port_name not in output_dict:
            output_dict[port_name] = [] # Initialize list if port doesn't exist

        # Ensure it's a list (might be corrupted or from old format)
        if not isinstance(output_dict[port_name], list):
             output_dict[port_name] = [output_dict[port_name]] # Convert single string to list

        if target_id in output_dict[port_name]:
            print(f"ℹ️ Info: Connection {nodes[source_id]['name']} [{port_name}] -> {nodes[target_id]['name']} already exists.")
        else:
            output_dict[port_name].append(target_id)
            print(f"✅ Connection added: {nodes[source_id]['name']} [{port_name}] -> {nodes[target_id]['name']}")

    visualize_graph() # Redraw graph

# --- Attach Handlers ---
node_type_dropdown.observe(handle_node_type_change, names='value')
add_update_node_button.on_click(on_add_update_node_clicked)
cancel_edit_button.on_click(on_cancel_edit_clicked)

load_node_button.on_click(on_load_node_clicked)
remove_node_button.on_click(on_remove_node_clicked)

source_node_dropdown.observe(update_source_ports, names='value')
add_connection_button.on_click(on_add_connection_clicked)

# --- Layout the UI ---
edit_node_section = widgets.VBox([
    widgets.HTML("<b>Manage Nodes:</b>"),
    widgets.HBox([select_node_edit_dropdown, load_node_button, remove_node_button]),
    edit_feedback_output
])

add_node_section = widgets.VBox([
    widgets.HTML("<hr><b>Add/Update Node Details:</b>"),
    node_type_dropdown, node_name_text,
    widgets.HTML("<i>Node Specific Options:</i>"), options_accordion,
    widgets.HBox([add_update_node_button, cancel_edit_button]), # Add/Update and Cancel side-by-side
    add_node_output
])

add_connection_section = widgets.VBox([
    widgets.HTML("<hr><b>Add Connection:</b>"),
    source_node_dropdown, source_port_dropdown, target_node_dropdown,
    add_connection_button,
    add_connection_output
])

# --- Initial Population ---
update_all_dropdowns()

# --- Display UI ---
display(widgets.VBox([
    edit_node_section,
    add_node_section,
    add_connection_section,
    widgets.HTML("<hr><b>Current Graph:</b>"),
    graph_output
]))

print("Node Definition, Editing, and Removal UI created.")

In [ ]:
# @title Play Mode Execution Logic and UI (Handles Multiple Connections)

play_mode_button = widgets.Button(description="🚀 Enter Play Mode", button_style='warning')
play_output = widgets.Output() # Area to display the interactive elements and results
full_log_output = widgets.Output() # Separate area for detailed execution logs

# --- Execution Function (Modified to handle list of next nodes) ---
active_executions = {} # Track ongoing executions if needed (more advanced)

def execute_node_flow(start_node_id, input_data=None):
    """Executes the node flow starting from a given node, handling branches."""
    queue = [(start_node_id, input_data)] # Queue stores (node_id, input_data) tuples
    visited_in_this_run = set() # Prevent infinite loops within a single trigger run
    run_id = str(uuid.uuid4())[:6] # ID for this specific execution run

    with full_log_output:
        print(f"\n--- Starting Execution Run {run_id} from Node {start_node_id} ---")

    processed_count = 0 # Safety break for potential infinite loops in logic
    max_processed = len(nodes) * 5 # Allow visiting nodes multiple times across branches, but cap it

    while queue and processed_count < max_processed:
        current_node_id, current_input = queue.pop(0)
        processed_count += 1

        # Basic cycle detection *within a single path* - might need more sophisitication
        # For simple trees/DAGs this is less critical, but good practice
        # Note: A proper cycle detection might track the path taken to reach a node.
        # This simple check prevents immediate re-execution in this specific run.
        # if (current_node_id, str(current_input)) in visited_in_this_run: # More robust check?
        #      with full_log_output:
        #         print(f"[{run_id}] Skipping node {current_node_id}, already processed with similar input in this run.")
        #      continue
        # visited_in_this_run.add((current_node_id, str(current_input)))


        if current_node_id not in nodes:
            with play_output: display(Markdown(f"⚠️ **Error:** Node `{current_node_id}` not found."))
            with full_log_output: print(f"[{run_id}] Error: Node {current_node_id} not found.")
            continue

        node_data = nodes[current_node_id]
        node_name = node_data['name']
        node_type = node_data['type']

        with full_log_output:
             print(f"[{run_id}] Executing Node: {node_name} ({current_node_id}), Type: {node_type}")

        output_data = None
        port_to_follow = None
        try:
            # --- Node Type Specific Logic (Mostly Unchanged) ---
            if node_type == 'Button':
                output_data = current_input
                port_to_follow = 'on_click'

            elif node_type == 'LLM Program':
                prompt_template = node_data['options'].get('prompt', '')
                prompt_text = prompt_template.replace('{input}', str(current_input) if current_input is not None else '')
                if client:
                    output_data = call_llm(prompt_text)
                else:
                    output_data = "Error: OpenAI client not initialized."
                    with play_output: display(Markdown(f"⚠️ **LLM Error:** OpenAI client not initialized."))
                port_to_follow = 'result'

            elif node_type == 'Show':
                text_template = node_data['options'].get('text_template', 'No text configured.')
                display_text = text_template.replace('{input}', str(current_input) if current_input is not None else '')
                with play_output:
                     display(Markdown(f"**{node_name}:**\n\n```\n{display_text}\n```"))
                output_data = display_text
                port_to_follow = None # Show node is an endpoint

            else:
                with play_output: display(Markdown(f"❓ **Unknown Node Type:** `{node_type}`"))
                with full_log_output: print(f"[{run_id}] Error: Unknown node type {node_type}")
                port_to_follow = None

            # --- Trigger Next Nodes (MODIFIED for list) ---
            if port_to_follow and port_to_follow in node_data.get('outputs', {}):
                next_node_ids = node_data['outputs'][port_to_follow] # This is now a LIST

                # Ensure it's a list for safety
                if isinstance(next_node_ids, str): next_node_ids = [next_node_ids]

                if isinstance(next_node_ids, list):
                    for next_node_id in next_node_ids: # Iterate through the list
                        if next_node_id:
                            with full_log_output:
                                print(f"[{run_id}] Queueing next node: {next_node_id} from port '{port_to_follow}'")
                            queue.append((next_node_id, output_data)) # Pass output as input to each next node
                # else: handle potential error/warning?

        except Exception as e:
             with play_output:
                 tb_str = traceback.format_exc()
                 display(Markdown(f"💥 **Execution Error** in node `{node_name}` ({current_node_id}):\n```\n{e}\n{tb_str}\n```"))
             with full_log_output:
                 print(f"[{run_id}] FATAL ERROR executing node {current_node_id}: {e}\n{traceback.format_exc()}")
             continue # Stop this branch on error

    if processed_count >= max_processed:
         with play_output: display(Markdown(f"⚠️ **Execution Limit Reached:** Processed {processed_count} nodes. Stopping run {run_id} to prevent potential infinite loop."))
         with full_log_output: print(f"[{run_id}] Execution limit ({max_processed}) reached. Stopping.")


# --- Play Mode Button Handler (No Change in core logic, just clears output) ---
def on_play_mode_clicked(b):
    """Sets up the Play Mode UI elements."""
    with play_output:
        clear_output(wait=True)
        display(Markdown("--- **Play Mode Activated** ---"))
        print("Looking for 'Button' nodes to start interaction...")

    with full_log_output:
        clear_output(wait=True)
        print("--- Play Mode Log ---")

    found_buttons = False
    for node_id, node_data in nodes.items():
        if node_data['type'] == 'Button':
            found_buttons = True
            button_widget = widgets.Button(description=f"▶️ {node_data['name']}", button_style='primary', tooltip=f"Trigger node {node_id}")

            def handle_play_button_click(b_instance, n_id=node_id): # Capture node_id
                with play_output: display(Markdown(f"▶️ *Clicked '{nodes[n_id]['name']}'...*"))
                # Crucially, call the MODIFIED execution function
                execute_node_flow(n_id, input_data=None)

            button_widget.on_click(handle_play_button_click)
            with play_output: display(button_widget)

    if not found_buttons:
         with play_output: display(Markdown("No 'Button' nodes found. Add a Button node to start."))

play_mode_button.on_click(on_play_mode_clicked)

# Display the Play Mode button and output areas
display(widgets.VBox([
    play_mode_button,
    widgets.HTML("<hr><b>Play Area:</b> (Click 'Enter Play Mode' first)"),
    play_output,
    widgets.HTML("<hr><b>Execution Log:</b>"),
    full_log_output
]))

print("Play Mode UI and Logic updated. Build/modify graph, then click 'Enter Play Mode'.")